<h2 style="text-align: center; font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: underline; text-transform: none; letter-spacing: 2px; color: darkred; background-color: #ffffff;"><br><br>NOVOZYMES ENZYME STABILITY PREDICTION<br><br> SURFACE AREA W/ BIOPYTHON<br></h2>

<h5 style="text-align: center; font-family: Verdana; font-size: 12px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: DARIEN SCHETTLER</h5>

<br>

---

<br>

<center><div class="alert alert-block alert-danger" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px;">🛑 &nbsp; WARNING:</b><br><br><b>THIS IS A WORK IN PROGRESS</b><br>
</div></center>


<center><div class="alert alert-block alert-warning" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px;">👏 &nbsp; IF YOU FORK THIS OR FIND THIS HELPFUL &nbsp; 👏</b><br><br><b style="font-size: 22px; color: darkorange">PLEASE UPVOTE!</b><br><br>This was a lot of work for me and while it may seem silly, it makes me feel appreciated when others like my work. 😅
</div></center>




In [2]:
print("\n... IMPORTS STARTING ...\n")

# Pip installs for competition specific stuff
!pip install -q biopandas
!pip install -q biopython
import numpy as np
import pandas as pd
from scipy import stats
import Bio
from Bio.PDB import PDBParser
from Bio.PDB.SASA import ShrakeRupley

print("\n\n... IMPORTS COMPLETE ...\n")


... IMPORTS STARTING ...



... IMPORTS COMPLETE ...



In [25]:
test_wt_pdb_path = "../input/novozymes-enzyme-stability-prediction/wildtype_structure_prediction_af2.pdb"
test_df = pd.read_csv("../input/novozymes-enzyme-stability-prediction/test.csv")
test_df_w_extra_info = pd.read_csv("../input/new-novoesp-train-and-test-data/new_test.csv")
ss_df = pd.read_csv("../input/novozymes-enzyme-stability-prediction/sample_submission.csv")
test_df = test_df.merge(test_df_w_extra_info[["seq_id", "edit_type", "edit_idx", "wildtype_aa", "mutant_aa"]], on="seq_id", how="left").fillna(-1)

# Wildtype test sequence 
wildtype_aa = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"

test_df

,seq_id,protein_sequence,pH,data_source,edit_type,edit_idx,wildtype_aa,mutant_aa
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16.0,L,E
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16.0,L,K
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,deletion,16.0,L,-1
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17.0,K,C
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17.0,K,F
...,...,...,...,...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15.0,A,I
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15.0,A,L
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15.0,A,N
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15.0,A,P


In [30]:
pdb_parser = PDBParser(QUIET=1)

def get_sr_sasa(pdb_path, return_struct=False, pdb_identifier="AF70", sr_n_points=250):
    sr = ShrakeRupley(n_points=sr_n_points)
    struct = pdb_parser.get_structure(pdb_identifier, pdb_path)
    sr.compute(struct, level="R")
    if return_struct:
        return struct
    else:
        return [x.sasa for x in struct.get_residues()]
    
sasa_by_residue = get_sr_sasa(test_wt_pdb_path)
residue_idx_to_sasa = {i:_sasa for i, _sasa in enumerate(sasa_by_residue)}
test_df["sasa"] = test_df["edit_idx"].apply(lambda x: residue_idx_to_sasa.get(x, 0.0))
test_df

,seq_id,protein_sequence,pH,data_source,edit_type,edit_idx,wildtype_aa,mutant_aa,stability_analysis,sasa
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16.0,L,E,43.819955,170.882740
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16.0,L,K,43.819955,170.882740
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,deletion,16.0,L,-1,43.973682,170.882740
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17.0,K,C,45.295068,71.392633
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17.0,K,F,43.819955,71.392633
...,...,...,...,...,...,...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15.0,A,I,44.307285,55.403077
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15.0,A,L,43.435792,55.403077
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15.0,A,N,43.435792,55.403077
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15.0,A,P,44.307285,55.403077


In [24]:
# This will score 0.397
ss_df["tm"] = test_df["sasa"]
ss_df.to_csv("sasa_submission.csv", index=False)

In [53]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
test_df["stability_analysis"] = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).instability_index())
test_df["aromaticity_analysis"] = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).aromaticity())
test_df["isoelectric_analysis"] = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).isoelectric_point())
test_df["charge_analysis"] = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).charge_at_pH(8.0))
test_df["helix_analysis"] = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).secondary_structure_fraction()[0])
test_df["turn_analysis"] = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).secondary_structure_fraction()[1])
test_df["sheet_analysis"] = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).secondary_structure_fraction()[2])
test_df["mec_analysis_1"] = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).molar_extinction_coefficient()[0])
test_df["mec_analysis_2"] = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).molar_extinction_coefficient()[1])
test_df["gravy_analysis"] = test_df["protein_sequence"].apply(lambda x: ProteinAnalysis(x).gravy())
test_df

,seq_id,protein_sequence,pH,data_source,edit_type,edit_idx,wildtype_aa,mutant_aa,stability_analysis,sasa,aromaticity_analysis,isoelectric_analysis,charge_analysis,helix_analysis,turn_analysis,sheet_analysis,mec_analysis_1,mec_analysis_2,gravy_analysis
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16.0,L,E,43.819955,170.882740,0.099548,8.751535,2.558953,0.230769,0.330317,0.180995,41940,42190,-0.771041
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,16.0,L,K,43.819955,170.882740,0.099548,9.024494,4.548770,0.230769,0.330317,0.176471,41940,42190,-0.772851
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,deletion,16.0,L,-1,43.973682,170.882740,0.100000,8.907484,3.558671,0.231818,0.331818,0.177273,41940,42190,-0.758636
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17.0,K,C,45.295068,71.392633,0.099548,8.699638,2.477663,0.235294,0.330317,0.180995,41940,42190,-0.709050
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,17.0,K,F,43.819955,71.392633,0.104072,8.761012,2.568572,0.239819,0.330317,0.180995,41940,42190,-0.707692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15.0,A,I,44.307285,55.403077,0.099548,8.907484,3.558671,0.239819,0.330317,0.176471,41940,42190,-0.725792
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15.0,A,L,43.435792,55.403077,0.099548,8.907484,3.558671,0.239819,0.330317,0.180995,41940,42190,-0.728959
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15.0,A,N,43.435792,55.403077,0.099548,8.907484,3.558671,0.235294,0.334842,0.176471,41940,42190,-0.761991
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,substitution,15.0,A,P,44.307285,55.403077,0.099548,8.907484,3.558671,0.235294,0.334842,0.176471,41940,42190,-0.753394
